# Vanilla **put** and **call** options pricing

---

We follow the exposition in Paul Wilmott's book. 

Definitions:
* $S_i$: underlying asset price at time $t_i$ ($i=0,...,N$, with $N$ the number of steps)
* $F$: strike price of the option
* $\delta t$: size of the single time step, defined by $\delta t = T/N$. $T$ is the time to maturity.
* $r$: continously compounded risk-free rate for the period $T$.

According to the binomial model, for each $i$,
$$ S_{i+1} = 
\begin{cases}
uS_i \text{ with probability } p \\
v S_i \text{ with probability } 1-p \\
\end{cases}.
$$
where $u>1$ and $v<1$.
The probability $p$ is actually irrelevant for the calculation.

The price for a **European option** at each time step is then given by
$$ V_i = e^{-r \delta t} \left[ p' V_{i+1}^+ + (1-p')(V_{i+1}^-) \right] $$
where $V_{i+1} ^{\pm}$ is the value of the option at the next time step supposing the underlying has moved to $S_{i+1} = uS_i$ or $S_{i+1} = vS_i$, respectively, and $p'$ is the _risk-neutral probability_  
$$ p' = \frac{e^{r\delta t} - v}{u-v}.$$
Note that at the last time step, $i=N$, the value of the option depends on the payout function. For calls $V_T =  \text{max}(S_T-F,0)$, for puts $V_T =  \text{max}(F-S_T,0)$.

Finally, for **American call options** the formula is modified to
$$V_i = \text{max} \left( e^{-r\delta t} \left[ p' V_{i+1}^+ + (1-p')(V_{i+1}^-) \right], S_i - F \right)$$
and for **American put options**
$$V_i = \text{max} \left( e^{-r\delta t} \left[ p' V_{i+1}^+ + (1-p')(V_{i+1}^-) \right], F - S_i \right).$$

---

Functions for the calculation of the price

In [1]:
import numpy as np
from typing import List, Dict

class OptionTypeInvalid(Exception):
    def __init__(self,  message="Option type invalid"):
        self.message = message
        super().__init__(self.message)

class OptionSpeciesInvalid(Exception):
    def __init__(self,  message="Option species invalid"):
        self.message = message
        super().__init__(self.message)

def option_payoff(option_type: 'str', asset_price: 'float', strike_price: 'float') -> float:
    """ Returns the value of this option give an asset price and strike price

    Attributes
    ----------
    option_type: str
        Either 'call' or 'put'
    asset_price: float
    strike_price: float

    Returns
    -------
    float
        The value of the option
    """
    if option_type == 'call':
        return max(0,asset_price-strike_price)
    elif option_type == 'put':
        return max(0, strike_price-asset_price)
    else:
        raise OptionTypeInvalid()
    

def option_prices(option_type: 'str', option_species : 'str', strike_price: 'float', stock_init_price: 'float', u: 'float', v: 'float', risk_free_rate: 'float' , tot_time: 'float', n_steps: 'int') -> List[Dict[int, float]]: 
    """ Returns the values of the option given the option details, the risk free rate, the maturity and the number of steps in the simulation

    Attributes
    ----------
    option_type: str
        Either 'call' or 'put'
    option_species: str
        Either 'european' (default) or 'american'
    strike_price: float
    stock_init_price: float
    u: float
    v: float
    risk_free_rate: float
    tot_time: float
    n_steps: int

    Returns
    -------
    List[Dict[str, int]]
        The values of the stock, of the option payoff and of the option at t_i
    """
    step_size =  tot_time/n_steps
    n_steps += 1
    stock_prices = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    option_payoffs = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    option_values = {i: [None for _ in range(i+1)] for i in range(n_steps)}
    discount_factor = np.exp(-risk_free_rate*step_size)
    pprime = (1/discount_factor-v)/(u-v)
    for i in range(n_steps):
        for j in range(i+1):
            stock_prices[i][j] = stock_init_price * (u**(i-j)) * (v**(j))
            option_payoffs[i][j] = option_payoff(option_type, stock_prices[i][j],strike_price)
    option_values[n_steps-1] = option_payoffs[n_steps-1]
    for i in range(n_steps):
        for j in range(n_steps-i-1):
            i_retro = n_steps-i-1
            if option_species == 'european':
                option_values[i_retro-1][j] = discount_factor*(pprime*option_values[i_retro][j] + (1-pprime)*option_values[i_retro][j+1])
            elif option_species == 'american':
                option_values[i_retro-1][j] = max( discount_factor*(pprime*option_values[i_retro][j] + (1-pprime)*option_values[i_retro][j+1]), 
                                                  option_payoff(option_type, stock_prices[i_retro-1][j], strike_price) )
            else:
                raise OptionSpeciesInvalid()
            
    return [stock_prices, option_payoffs, option_values]



--- 

## Examples

Wilmott book example (result matches the book)

In [7]:
u = 1.0604
v=  0.9431
strike_price = 100
stock_init_price = 100
r = 0.1
t_tot = 1/3 # 1/3 in years, so 4 months
n_steps =  4 # 4 steps, so delta_t = t_tot/4 = 1 month

stock_prices, option_payoffs, option_values = option_prices(option_type='call', option_species= 'european', strike_price = strike_price, stock_init_price = stock_init_price, u=u,v=v, risk_free_rate=r, tot_time = t_tot, n_steps=n_steps)

print('Stock prices')
for step, prices in stock_prices.items():
    print(step, ':', prices)
print('Option payoffs')
for step, prices in option_payoffs.items():
    print(step, ':', prices)
print('Option values')
for step, prices in option_values.items():
    print(step, ':', prices)

Stock prices
0 : [100.0]
1 : [106.04, 94.31]
2 : [112.444816, 100.006324, 88.94376100000001]
3 : [119.23648288640001, 106.04670596960001, 94.31596416440001, 83.88286099910002]
4 : [126.43836645273856, 112.45192701016386, 100.01264839992977, 88.94938580344567, 79.10992620825122]
Option payoffs
0 : [0]
1 : [6.040000000000006, 0]
2 : [12.444816000000003, 0.006324000000006436, 0]
3 : [19.23648288640001, 6.046705969600012, 0, 0]
4 : [26.438366452738563, 12.451927010163857, 0.01264839992977329, 0, 0]
Option values
0 : [6.136930000932622]
1 : [9.449690650117864, 2.0972146854304263]
2 : [14.097670617838267, 3.797587007438228, 0.003851269930354975]
3 : [20.066353622512416, 6.87657670571242, 0.006979427076532848, 0.0]
4 : [26.438366452738563, 12.451927010163857, 0.01264839992977329, 0, 0]
